In [3816]:
import csv
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import openpyxl
import os
from openpyxl import Workbook
from pandas import DataFrame


In [3817]:
#MOM CHANGE HERE ONLY !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


#Input CV arguments ["file1","file2","file3","file4"]
file_list = ["2-FeAc-111920-A5_Cond CV-1.txt", "2-FeAc-111920-A5_Cond CV-2.txt", "2-FeAc-111920-A5_Cond CV-3.txt", "2-FeAc-111920-A5_CV aft ORR-1.txt"]
CV_RHE = 0.217
Area = 0.237582944


#Input ORR arguments
orr_list = ["FeAc-111920-A8_SCV_ORR-1.txt","FeAc-111920-A8_SCV_ORR-2.txt"]
ORR_RHE = 0.225
Eff = 0.378

In [3818]:
def clear_prev():
    if os.path.exists("Test.xlsx"):
        os.remove("Test.xlsx")
    if os.path.exists("Data.xlsx"):
        os.remove("Data.xlsx")
    

In [3819]:
clear_prev()

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'Data.xlsx'

In [3783]:
def line_prepender(filename, line):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(line.rstrip('\r\n') + '\n' + content)

In [3784]:
def process_CV(filename,RHE,Area):
    #delete existing
    before = []
    data = []
    if os.path.exists("CV.csv"):
        os.remove("CV.csv")

    #read data into array
    with open(filename, mode="r", encoding="utf-8") as f:
        before.append(f.readline())
        f.readline()
        before.append(f.readline())
        for line in f:
            if "Potential" in line:
                break
        f.readline()
        for line in f:
            data.append(np.array(line.split())) 
   
    np_data = np.float64(data)
    np_data = np_data[-2000:,:]
    #add two extra columns
    V = np_data[:,0].copy()
    V += RHE
    np_data = np.insert(np_data,2, V, axis=1)

    I = np_data[:,1].copy()
    I = I * 1000 /Area

    np_data = np.insert(np_data,3, I, axis=1)
    return before, np_data

In [3785]:

def combine_data(file_list,RHE,Area):
    data_list = []
    np_data = np.array([])
    arr1 = np.array([])
    arr2 = np.array([])
    for i in range(len(file_list)):
        if(i==0):
            before, np_data = process_CV(file_list[i],RHE,Area)
            data_list.append(np_data)
            arr1 = np.append(arr1, before[0])
            #arr1[0].rstrip("\n")
            arr2 = np.append(arr2, before[1])
            for i in range(3):
                arr1 = np.append(arr1, "")
                arr2 = np.append(arr2, "")
        else:
            temp_before, temp = process_CV(file_list[i],RHE,Area)
            data_list.append(temp)
            np_data = np.concatenate([np_data, temp],axis=1)
            temp_arr1 = np.array([])
            temp_arr2 = np.array([])
            temp_arr1 = np.append(temp_arr1, temp_before[0])
           # temp_arr1[0].rstrip("\n")
            temp_arr2 = np.append(temp_arr2, temp_before[1])
            for i in range(3):
                temp_arr1 = np.append(temp_arr1, "")
                temp_arr2 = np.append(temp_arr2, "")
            arr1 = np.append(arr1, temp_arr1)
            arr2 = np.append(arr2, temp_arr2)
    return data_list, arr1, arr2 , np_data

In [3786]:
def create_CSV(file_list,RHE,Area):
    #create headers
    headers1 = []
    headers1.append("Potential/V")
    headers1.append("Current/I")
    headers1.append("Potential/V vs RHE")
    headers1.append("Current Density/mA/cm^2")
    headers2 = headers1 * len(file_list)
    headers = ','.join(headers2)
   # print(len(headers2))
    data_list, arr1, arr2, np_data = combine_data(file_list,RHE,Area)
    for i in range(len(arr1)):
        arr1[i]= arr1[i].rstrip("\n")
    for i in range(len(arr2)):
        arr2[i] = arr2[i].rstrip("\n")
    #process intial date and file name
    string1 = ','.join(arr1)
    string2 = ','.join(arr2)
    string1 = string1.rstrip(",")
    string2 = string2.rstrip(",")
    #save to csv file
    np.savetxt("CV.csv", np_data, delimiter = ",", fmt='%s')
    #prepend info
    line_prepender("CV.csv",headers)
    df = pd.read_csv("CV.csv")
    line_prepender("CV.csv", string2)
    line_prepender("CV.csv", string1)
    return df,data_list

In [3787]:
cv_df,data_list = create_CSV(file_list,CV_RHE,Area)
cv_df

,Potential/V,Current/I,Potential/V vs RHE,Current Density/mA/cm^2,Potential/V.1,Current/I.1,Potential/V vs RHE.1,Current Density/mA/cm^2.1,Potential/V.2,Current/I.2,Potential/V vs RHE.2,Current Density/mA/cm^2.2,Potential/V.3,Current/I.3,Potential/V vs RHE.3,Current Density/mA/cm^2.3
0,0.775,0.002877,0.992,12.109455,0.775,0.002840,0.992,11.953720,-0.225,-0.003359,-0.008,-14.138220,-0.225,-0.004089,-0.008,-17.210831
1,0.774,0.002604,0.991,10.960383,0.774,0.002570,0.991,10.817275,-0.224,-0.003084,-0.007,-12.980730,-0.224,-0.003823,-0.007,-16.091222
2,0.773,0.002350,0.990,9.891282,0.773,0.002308,0.990,9.714502,-0.223,-0.002820,-0.006,-11.869539,-0.223,-0.003554,-0.006,-14.958986
3,0.772,0.002109,0.989,8.876900,0.772,0.002062,0.989,8.679074,-0.222,-0.002575,-0.005,-10.838320,-0.222,-0.003320,-0.005,-13.974067
4,0.771,0.001884,0.988,7.929862,0.771,0.001842,0.988,7.753082,-0.221,-0.002346,-0.004,-9.874446,-0.221,-0.003092,-0.004,-13.014402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.770,0.002927,0.987,12.319908,0.770,0.002900,0.987,12.206263,-0.220,-0.003370,-0.003,-14.184520,-0.220,-0.004132,-0.003,-17.391821
1996,0.771,0.002941,0.988,12.378835,0.771,0.002905,0.988,12.227309,-0.221,-0.003373,-0.004,-14.197147,-0.221,-0.004141,-0.004,-17.429702
1997,0.772,0.002948,0.989,12.408298,0.772,0.002911,0.989,12.252563,-0.222,-0.003377,-0.005,-14.213983,-0.222,-0.004133,-0.005,-17.396030
1998,0.773,0.002955,0.990,12.437762,0.773,0.002914,0.990,12.265190,-0.223,-0.003382,-0.006,-14.235029,-0.223,-0.004121,-0.006,-17.345521


In [3788]:
# def create_graph(data_list):
#     plt.figure()
    
#     last_cycle0 = data_list[0][:,2:4]
#     plt.plot(last_cycle0[:,0], last_cycle0[:,1], label = "CV1")

#     last_cycle1 = data_list[1][:,2:4]
#     plt.plot(last_cycle1[:,0], last_cycle1[:,1], label = "CV2")

#     last_cycle2 = data_list[2][:,2:4]
#     plt.plot(last_cycle2[:,0], last_cycle2[:,1], label = "CV3")

#     last_cycle3 = data_list[3][:,2:4]
#     plt.plot(last_cycle3[:,0], last_cycle3[:,1], label = "AFTER ORR-1")

#     plt.title("CV")
#     plt.xlabel(headers1[2])
    
#     plt.legend(loc="upper left")
#     plt.savefig("CV.png", dpi = 150)


In [3789]:
# create_graph(data_list)

In [3790]:
#create Data.xlsx
# wb = Workbook()
# ws = wb.active
# with open('CV.csv', 'r') as f:
#     for row in csv.reader(f):
#         ws.append(row)
# wb.save('Data.xlsx')



In [3791]:
#transfer PNG of graph to excel
#wb = openpyxl.load_workbook('Test.xlsx')

#Update sheet name
# ss_sheet = wb['Sheet1']
# ss_sheet.title = 'CV'
# wb.save("Test.xlsx")

#add png
# sheet = wb['CV']
# img = openpyxl.drawing.image.Image('CV.png')
# sheet.add_image(img, 'S11')
# wb.save('Test.xlsx')

In [3792]:
#add sheet
# wb.create_sheet("ORR")
# wb.save('Test.xlsx')

In [3793]:
pot_val = cv_df.iloc[:,10].to_numpy()
dens_val = cv_df.iloc[:,11].to_numpy()
pot_val

start_point02 =-1
end_point05 =-1

start_point04=-1
end_point01=-1
for i in range(len(pot_val)):
    if(pot_val[i]==0.2 and pot_val[i+1]>pot_val[i]):
        start_point02=i
        for j in range(i,len(pot_val)):
            if(pot_val[j]==0.5):
                end_point05 = j
                break
        break
for i in range(len(pot_val)):
    if(pot_val[i]==0.4 and pot_val[i+1]<pot_val[i]):
        start_point04 = i
        for j in range(i,len(pot_val)):
            if(pot_val[j]==0.1):
                end_point01 =j
                break
        break
print(start_point02, end_point05)
print(start_point04, end_point01)

208 508
1592 1892


In [3794]:

calc_25 = np.column_stack((pot_val[start_point02-2:end_point05+1], dens_val[start_point02-2:end_point05+1]))
calc_41 = np.column_stack((pot_val[start_point04-2:end_point01+1], dens_val[start_point04-2:end_point01+1]))


point_charge25 = np.zeros(len(calc_25)-2)
point_charge41 = np.zeros(len(calc_41)-2)
scan_rate = 0.1
for i in range(len(calc_25)):
    if(i==0 or i==1):
        continue
    else:
        point_charge25[i-2] = (calc_25[i-1][1] + calc_25[i-2][1])/2 * Area *(calc_25[i-1][0] - calc_25[i-2][0]) / scan_rate
        point_charge41[i-2] = (calc_41[i-1][1] + calc_41[i-2][1])/2 * Area *(calc_41[i-1][0] - calc_41[i-2][0]) / scan_rate
point_charge41.shape

(301,)

In [3795]:
cv_names = ["Scan Rate (V/s)", "Double layer charge neg (mAs)", "Double layer charge pos (mAs)", "Total Charge (mAs)", "delta V (V)", "Capacitance (mF)"]


In [3796]:
#double layer charge neg
neg_charge = sum(point_charge41)
#double layer charge positive
pos_charge = sum(point_charge25)
#double  layer charge total
total_charge = neg_charge+ pos_charge
print(neg_charge,pos_charge,total_charge)
#Voltage Difference
deltaV = 0.3
#Capacitance
C = total_charge/deltaV/2

8.930235000000001 7.6479399999999975 16.578174999999998


In [3797]:
cv_calc = np.array([])
cv_calc = np.append(cv_calc,scan_rate)
cv_calc = np.append(cv_calc,neg_charge)
cv_calc = np.append(cv_calc,pos_charge)
cv_calc = np.append(cv_calc,total_charge)
cv_calc = np.append(cv_calc,deltaV)
cv_calc = np.append(cv_calc,C)

In [3798]:
def process_ORR(filename, ORR_RHE,Area, Eff):
#delete existing
    before = []
    data = []
    if os.path.exists("ORR.csv"):
        os.remove("ORR.csv")

    #read data into array
    with open(filename, mode="r", encoding="utf-8") as f:
        before.append(f.readline())
        f.readline()
        before.append(f.readline())
        for line in f:
            if "Potential" in line:
                break
        f.readline()
        for line in f:
            data.append(np.array(line.split())) 
        np_data = np.float64(data)
        
        P = np_data[:,0].copy()
        P+= ORR_RHE
        np_data = np.insert(np_data,3, P, axis=1)

        I1 = np_data[:,1].copy()
        I1_temp = I1*1000/Area
        np_data = np.insert(np_data,4, I1_temp, axis=1)

        I2 = np_data[:,2].copy()

        Y = -(200*(I2/Eff))/(I1+I2/Eff)
        np_data = np.insert(np_data,5, Y, axis=1)


   
    #add two extra columns
    return np_data

In [3799]:
def calc_avg_pot(orr_data):

    d={}
    pot = orr_data[:,3]
    I1 = orr_data[:,4]
    YH202 = orr_data[:,5]
    for index in range(len(pot)):
        if pot[index] not in d.keys():
            d[pot[index]] = [index]
        else:
            d[pot[index]].append(index) 
    v_avg = []
    i1_avg = []
    yh_avg = []
    for key,value in d.items():
        v_avg.append(key)
        avg1= 0
        avg2= 0
        for i in range(len(value)):
            avg1 += I1[value[i]]
            avg2 += YH202[value[i]]
        avg1/=len(value)
        avg2/=len(value)
        i1_avg.append(avg1)
        yh_avg.append(avg2)
    
    v_avg = np.float64(v_avg)
    final_i1 = np.float64(i1_avg)
    final_yh = np.float64(yh_avg)
    final_list = np.column_stack((v_avg, final_i1))
    final_list = np.column_stack((final_list, final_yh))
    return final_list

In [3800]:

def combine_ORR(orr_list,ORR_RHE,Area, Eff):
    calc_list = []
    for i in range(len(orr_list)):
            orr_data = process_ORR(orr_list[i],ORR_RHE,Area, Eff)
            calc_list.append(orr_data)
    df_list =[]
    for i in range(len(calc_list)):
        temp_df = pd.DataFrame(calc_list[i])  #dataframe from original data
        final_list = calc_avg_pot(calc_list[i]) #calculate 3 extra columns
        temp_df1 = pd.DataFrame(final_list) #dataframe from 3 extra columns
        orr = pd.concat([temp_df,temp_df1],ignore_index=True,axis=1)  #combine
        df_list.append(orr)
    if(len(orr_list)==1):
        orr_df = df_list[0]
    elif(len(orr_list)==2):
        orr_df = pd.concat([df_list[0],df_list[1]], ignore_index = True, axis=1) 
    return orr_df

In [3801]:

orr_df = combine_ORR(orr_list, ORR_RHE, Area, Eff)

column_names = ["Potential/V", "I1/A", "I2/A", "Potential/V vs RHE", "I1 Current Density/mA/cm^2", "YH202/%", "Avg Potential/V vs RHE", "Avg I1/mA/cm^2", "Avg YH202/%"] 
orr_df.columns = column_names*2

orr_df

,Potential/V,I1/A,I2/A,Potential/V vs RHE,I1 Current Density/mA/cm^2,YH202/%,Avg Potential/V vs RHE,Avg I1/mA/cm^2,Avg YH202/%,Potential/V,I1/A,I2/A,Potential/V vs RHE,I1 Current Density/mA/cm^2,YH202/%,Avg Potential/V vs RHE,Avg I1/mA/cm^2,Avg YH202/%
0,0.755,0.000047,0.000016,0.98,0.197194,-95.794853,0.98,0.197194,-95.794853,-0.225,-0.000895,0.000024,0.00,-3.768368,14.965220,0.00,-3.768368,14.965220
1,0.735,0.000026,0.000016,0.96,0.108888,-123.955727,0.96,0.116359,-116.827978,-0.205,-0.000864,0.000022,0.02,-3.636204,14.729371,0.02,-3.627996,13.603451
2,0.715,0.000016,0.000016,0.94,0.067429,-144.433086,0.94,0.087485,-130.673436,-0.185,-0.000851,0.000022,0.04,-3.582749,14.385488,0.04,-3.588852,13.313118
3,0.695,0.000009,0.000016,0.92,0.037671,-164.431665,0.92,0.063241,-145.368961,-0.165,-0.000847,0.000021,0.06,-3.565071,14.060494,0.06,-3.577067,13.049576
4,0.675,0.000001,0.000016,0.90,0.005876,-193.468273,0.90,0.033243,-168.388861,-0.145,-0.000844,0.000021,0.08,-3.553285,13.801994,0.08,-3.569701,12.848835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0.655,0.000005,0.000014,0.88,0.019513,-177.551086,NaN,NaN,NaN,-0.125,-0.000849,0.000018,0.10,-3.572226,11.825740,NaN,NaN,NaN
94,0.675,0.000014,0.000014,0.90,0.060610,-143.309448,NaN,NaN,NaN,-0.145,-0.000852,0.000018,0.08,-3.586116,11.895676,NaN,NaN,NaN
95,0.695,0.000021,0.000014,0.92,0.088811,-126.306258,NaN,NaN,NaN,-0.165,-0.000853,0.000018,0.06,-3.589062,12.038658,NaN,NaN,NaN
96,0.715,0.000026,0.000014,0.94,0.107541,-116.913786,NaN,NaN,NaN,-0.185,-0.000854,0.000019,0.04,-3.594955,12.240747,NaN,NaN,NaN


In [3802]:
#pd.to_numeric(df.iloc[:,0], errors='ignore')
#cv_df
n=0
for i in range(len(file_list)-1):
    if i==3:
        cv_df.insert(4*(i+1)+n,np.nan, np.nan, allow_duplicates=True)
    cv_df.insert(4*(i+1)+n,np.nan, np.nan, allow_duplicates=True)
    cv_df.insert(4*(i+1)+n,np.nan, np.nan, allow_duplicates=True)
    n+=2

writer = pd.ExcelWriter("Data.xlsx")
cv_df.to_excel(writer, sheet_name='CV')

#Pandas to Excel

workbook = writer.book
worksheet = writer.sheets['CV']
chart = workbook.add_chart({'type':'scatter', 'subtype': 'straight'})
#[sheetname, first_row, first_col, last_row, last_col]. 
n=0
for i in range(len(file_list)):
    chart.add_series({
        'name': "CV" + str(i+1),
        'categories': ['CV',1,3+4*i+n,2000,3+4*i+n],
        'values': ['CV',1,4+4*i+n,2000,4+4*i+n],
    })
    n+=2
chart.set_title({'name':'CV data'})
chart.set_x_axis({'name':headers1[2]})
chart.set_y_axis({'name':headers1[3]})
worksheet.insert_chart('Y4',chart)

#orr_df
n=0
for i in range(len(orr_list)-1):
    orr_df.insert(9*(i+1)+n,np.nan, np.nan, allow_duplicates=True)
    orr_df.insert(9*(i+1)+n,np.nan, np.nan, allow_duplicates=True)
    n+=1
orr_df.to_excel(writer,sheet_name="ORR")

#workbook = xlsxwriter.Workbook("Test.xlsx", {'strings_to_numbers': True})
# last_cycle0 = df.iloc[2:2002,2:4]
# last_cycle0 


worksheet = writer.sheets['ORR']
orr_chart1 = workbook.add_chart({'type':'scatter', 'subtype': 'straight_with_markers'})
n=0
#[sheetname, first_row, first_col, last_row, last_col]. 
for i in range(len(orr_list)):
    orr_chart1.add_series({
        'name': "ORR-" + str(i+1),
        'categories': ['ORR',1,4+9*i+n,98,4+9*i+n],
        'values': ['ORR',1,5+9*i+n,98,5+9*i+n],
        'marker': {'type': 'diamond', 'size': 4},
    })
    n+=2
orr_chart1.set_title({'name':'ORR-SCV'})
orr_chart1.set_x_axis({'name':column_names[3]})
orr_chart1.set_y_axis({'name':column_names[4]})
worksheet.insert_chart('N10',orr_chart1)

orr_chart2 = workbook.add_chart({'type':'scatter', 'subtype': 'straight_with_markers'})
n=0
#[sheetname, first_row, first_col, last_row, last_col]. 
for i in range(len(orr_list)):
    orr_chart2.add_series({
        'name': "ORR-" + str(i+1),
        'categories': ['ORR',1,4+9*i+n,98,4+9*i+n],
        'values': ['ORR',1,6+9*i+n,98,6+9*i+n],
        'marker': {'type': 'diamond', 'size': 4},
    })
    n+=2
orr_chart2.set_title({'name':'H202 Yield'})
orr_chart2.set_x_axis({'name':column_names[3]})
orr_chart2.set_y_axis({'name':column_names[5]})
worksheet.insert_chart('N25',orr_chart2)


orr_chart3 = workbook.add_chart({'type':'scatter', 'subtype': 'straight_with_markers'})
n=0
#[sheetname, first_row, first_col, last_row, last_col]. 
for i in range(len(orr_list)):
    orr_chart3.add_series({
        'name': "ORR-" + str(i+1),
        'categories': ['ORR',1,7+9*i+n,50,7+9*i+n],
        'values': ['ORR',1,8+9*i+n,50,8+9*i+n],
        'marker': {'type': 'diamond', 'size': 4},
    })
    n+=2
orr_chart3.set_title({'name':'ORR-SCV Avg'})
orr_chart3.set_x_axis({'name':column_names[6]})
orr_chart3.set_y_axis({'name':column_names[7]})
worksheet.insert_chart('N40',orr_chart3)

orr_chart4 = workbook.add_chart({'type':'scatter', 'subtype': 'straight_with_markers'})
n=0
#[sheetname, first_row, first_col, last_row, last_col]. 
for i in range(len(orr_list)):
    orr_chart4.add_series({
        'name': "ORR-" + str(i+1),
        'categories': ['ORR',1,7+9*i+n,50,7+9*i+n],
        'values': ['ORR',1,9+9*i+n,50,9+9*i+n],
        'marker': {'type': 'diamond', 'size': 4},
    })
    n+=2
orr_chart4.set_title({'name':'H202 Avg Yield'})
orr_chart4.set_x_axis({'min': 0, 'max': 0.8,'name':column_names[6]})
orr_chart4.set_y_axis({'min': 0, 'max': 30,'name':column_names[8]})
worksheet.insert_chart('N55',orr_chart4)


#potential/V vs RHE and I1 current density + avg
#potential/V vs RHE and YH202/% + avg
#4 graphs

In [3803]:
#Limiting Current Density
#Current Density at 0.8 V
#Kinetic Current Density at 0.8 V
#Mass Activity
#1/2 Limiting Current Density
#Slope
#Y-Intercept
#Half Wave Potential
#H202 Yield at 0.4 V

#["Potential/V", "I1/A", "I2/A", "Potential/V vs RHE", "I1 Current Density/mA/cm^2", "YH202/%", "Avg Potential/V vs RHE", "Avg I1/mA/cm^2", "Avg YH202/%"] 

In [3804]:
#Limiting Current Density

avg_pot = orr_df.loc[:,"Avg Potential/V vs RHE"].to_numpy()[:50]
avg_I1 =  orr_df.loc[:,"Avg I1/mA/cm^2"].to_numpy()[:50]

limit_current = []
for i in range(len(avg_pot[0])):
    avg = 0
    points = 0  
    for j in range(len(avg_pot[:,i])):
        if(avg_pot[:,i][j]>=0.1 and avg_pot[:,i][j]<=0.4):
            avg+= avg_I1[:,i][j]
            points+=1
    avg /=points
    limit_current.append(avg)
limit_current = -np.array(limit_current)
limit_current

array([3.49124125, 3.46723648])

In [3805]:
#Current Density at 0.8 V
avg_dict1 = {}
avg_dict2 = {}
for j in range(len(avg_pot[:,0])):
    avg_dict1[np.round(avg_pot[:,0][j],2)] = avg_I1[:,0][j]
for j in range(len(avg_pot[:,1])):
    avg_dict2[np.round(avg_pot[:,1][j],2)] = avg_I1[:,1][j]
curr_dens = -np.array([avg_dict1[0.8], avg_dict2[0.8]])
curr_dens


array([0.72458905, 0.65998004])

In [3806]:
#Kinetic Current Density
kinetic_current = limit_current * curr_dens / (limit_current - curr_dens)
kinetic_current

array([0.91435967, 0.81513994])

In [3807]:
#Mass Activity
catalyst_loading = 0.1425498
mass_activity = kinetic_current * Area / catalyst_loading
mass_activity

array([1.52393242, 1.35856625])

In [3808]:
#1/2 Limiting Current Density
half_limit_curr = limit_current/2
half_limit_curr

array([1.74562063, 1.73361824])

In [3809]:
regress_points1 = np.array([[0,0]])
regress_points2 = np.array([[0,0]])
for i in range(len(avg_pot[0])):
    for j in range(len(avg_pot[:,i])-1):
        if(((-half_limit_curr[i])>=avg_I1[:,i][j] and (-half_limit_curr[i])<=avg_I1[:,i][j+1]) or ((-half_limit_curr[i])<=avg_I1[:,i][j] and (-half_limit_curr[i])>=avg_I1[:,i][j+1])):
            temp = np.array([[avg_pot[:,i][j], avg_I1[:,i][j]]])
            temp1 = np.array([[avg_pot[:,i][j+1], avg_I1[:,i][j+1]]])
            if(i==0):
                regress_points1 = np.append(regress_points1, temp, axis =0)
                regress_points1 = np.append(regress_points1, temp1, axis =0)
                regress_points1 = regress_points1[1:]
                break
            else:
                regress_points2 = np.append(regress_points2, temp, axis=0)
                regress_points2 = np.append(regress_points2, temp1, axis=0)
                regress_points2 = regress_points2[1:]
                break
regress_points1

array([[ 0.76      , -1.51694391],
       [ 0.74      , -1.87050464]])

In [3810]:
from sklearn.linear_model import LinearRegression

In [3811]:
x1 = regress_points1[:,0].reshape((-1,1))
y1 =regress_points1[:,1]

x2 = regress_points2[:,0].reshape((-1,1))
y2 = regress_points2[:,1]

model = LinearRegression().fit(x1,y1)
slope = model.coef_
slope = slope[0]
y_intercept = model.intercept_
x = (-half_limit_curr[0] -y_intercept)/slope 

model2 = LinearRegression().fit(x2,y2)
slope2 = model2.coef_
slope2 = slope2[0]
y_intercept2 = model2.intercept_
x2 = (-half_limit_curr[1] -y_intercept2)/slope2 

slopes = np.array([slope, slope2])
y_intercepts = np.array([y_intercept, y_intercept2])
half_wave_pot = np.array([x,x2])

In [3812]:
#H202 Yield at 0.4 V
avg_Y =  orr_df.loc[:,"Avg YH202/%"].to_numpy()[:50]

avg_Y1 = {}
avg_Y2 = {}
for j in range(len(avg_pot[:,0])):
    avg_Y1[np.round(avg_pot[:,0][j],2)] = avg_Y[:,0][j]
for j in range(len(avg_pot[:,1])):
    avg_Y2[np.round(avg_pot[:,1][j],2)] = avg_Y[:,1][j]
Y_val = np.array([avg_Y1[0.4], avg_Y2[0.4]])
Y_val

array([14.69708909, 14.82684398])

In [3813]:
final_arr = np.array([[0,0]])
final_arr = np.append(final_arr,[limit_current],axis =0 )
final_arr = np.append(final_arr,[curr_dens],axis =0 )
final_arr = np.append(final_arr,[kinetic_current],axis =0 )
final_arr = np.append(final_arr,[mass_activity],axis =0 )
final_arr = np.append(final_arr,[half_limit_curr],axis =0 )
final_arr = np.append(final_arr,[slopes],axis =0 )
final_arr = np.append(final_arr,[y_intercepts],axis =0 )
final_arr = np.append(final_arr,[half_wave_pot],axis =0 )
final_arr = np.append(final_arr,[Y_val],axis =0 )
final_arr = final_arr[1:]

row_names = ["Limiting Current Density (mA/cm^2)", "Current Density at 0.8 V (mA/cm^2)","Kinetic Current Density at 0.8 V (mA/cm^2)","Mass Activity (mA/mg)","1/2 Limiting Current Density (mA/cm^2)","Slope","Y-Intercept", "Half Wave Potential (V vs RHE)","H202 Yield at 0.4 V (%)"]
df_calc = pd.DataFrame(final_arr, index = row_names)
col_names = ["ORR1","ORR2"] 

df_calc.columns = col_names
print(df_calc)

worksheet = writer.sheets['ORR']
df_calc.to_excel(writer,sheet_name="ORR", startcol = 23, startrow = 8)


                                                 ORR1       ORR2
Limiting Current Density (mA/cm^2)           3.491241   3.467236
Current Density at 0.8 V (mA/cm^2)           0.724589   0.659980
Kinetic Current Density at 0.8 V (mA/cm^2)   0.914360   0.815140
Mass Activity (mA/mg)                        1.523932   1.358566
1/2 Limiting Current Density (mA/cm^2)       1.745621   1.733618
Slope                                       17.678037  17.593856
Y-Intercept                                -14.952252 -14.803672
Half Wave Potential (V vs RHE)               0.747064   0.742876
H202 Yield at 0.4 V (%)                     14.697089  14.826844


In [3814]:
worksheet = writer.sheets['CV']
workbook = writer.book

df_25 =pd.DataFrame(point_charge25)
df_41 = pd.DataFrame(point_charge41)
df_25.to_excel(writer,sheet_name="CV", index = False, header=
["Point Charge Pos"], startrow =start_point02, startcol = 17)

df_41.to_excel(writer, sheet_name="CV",index = False,header=
["Point Charge Neg"], startrow = start_point04, startcol =17)


In [3815]:
print(cv_names)
df_cv_calc = pd.DataFrame(cv_calc,index = cv_names)
df_cv_calc.to_excel(writer, sheet_name = "CV", header = ["Capacitance Calculation"],startrow = 21, startcol = 26)
writer.save()

['Scan Rate (V/s)', 'Double layer charge neg (mAs)', 'Double layer charge pos (mAs)', 'Total Charge (mAs)', 'delta V (V)', 'Capacitance (mF)']
